# Begin model training

## Load libraries

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import os

## Setup Colab

In [2]:
COLAB = 'google.colab' in str(get_ipython())

if COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT = '/content/drive/MyDrive/fyp/collabs/'
else:
    ROOT = os.path.join(os.getcwd(), '..', '..') 

## Load Colab libraries

In [3]:
if COLAB:
    !pip install pickle5
    import pickle5 as pickle

## Load custom modules

In [4]:
from nnssa.constants import *
from nnssa.evaluate import evaluate

## Set random seed for reproducability

In [5]:
SEED = 42
np.random.seed(SEED)

## Load TensorFlow modules

In [6]:
# %tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [7]:
def get_metrics():
    return [
        tf.keras.metrics.BinaryAccuracy(name='accuracy', threshold=0.15),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
    ]

## Enable TPU

In [8]:
if ('COLAB_TPU_ADDR' in os.environ.keys()):
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_host(resolver.master())
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

## Progress bars!

In [9]:
from tqdm import tqdm
tqdm.pandas()

/Users/admin/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Load dataset

In [10]:
if COLAB:
    harmonix_beats = pickle.load(open(os.path.join(ROOT, SUB_DIVS_DIR, 'bars', 'harmonix.p'), 'rb'))
else:
    harmonix_beats = pd.read_pickle(os.path.join(ROOT, SUB_DIVS_DIR, 'bars', 'harmonix.p'))
harmonix_beats = harmonix_beats.head(100)
harmonix_beats.head()

,File,Sub_Divisions,Binary_Labels,Weighted_Labels,Weights,IDS,Beat_times,Labels,BPM
0,0001_12step,bars/harmonix/0001_12step.npy,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, ...","[0001_12step, 0001_12step, 0001_12step, 0001_1...","[0.0, 0.5309729999999999, 1.0619459999999998, ...","[0.0, 8.495567999999999, 25.486704, 42.4753280...",113
1,0003_6foot7foot,bars/harmonix/0003_6foot7foot.npy,"[1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0003_6foot7foot, 0003_6foot7foot, 0003_6foot7...","[2.857108, 3.571394, 4.28568, 4.99996600000000...","[2.857108, 8.571396, 31.428548, 37.14283599999...",84
2,0004_abc,bars/harmonix/0004_abc.npy,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0004_abc, 0004_abc, 0004_abc, 0004_abc, 0004_...","[2.666656, 3.238084, 3.952369, 4.597529, 5.242...","[2.666656, 28.300542999999998, 58.263180000000...",94
3,0006_aint2proud2beg,bars/harmonix/0006_aint2proud2beg.npy,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0006_aint2proud2beg, 0006_aint2proud2beg, 000...","[0.0, 0.572203, 1.144406, 1.716609, 2.288812, ...","[0.0, 27.4652, 45.203726, 63.518522999999995, ...",105
4,0008_america,bars/harmonix/0008_america.npy,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 0.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...","[3.0, 0.5, 1.0, 0.5, 3.0, 0.5, 1.0, 1.0, 1.0, ...","[0008_america, 0008_america, 0008_america, 000...","[3.871208, 4.359011, 4.846814, 5.338616, 5.830...","[3.871208, 10.56504, 33.217138, 56.85190400000...",136


## Load Mel Spectrograms

In [11]:
def load_np(file):
    return np.load(os.path.join(ROOT, SUB_DIVS_DIR, file), allow_pickle=True)

In [12]:
harmonix_beats['Sub_Divisions'] = harmonix_beats['Sub_Divisions'].progress_map(load_np)

100%|██████████| 100/100 [00:00<00:00, 166.79it/s]


In [13]:
# harmonix_beats['Oversamples'] = harmonix_beats['Oversamples'].progress_map(load_np)

In [14]:
X = harmonix_beats.copy()
y = harmonix_beats['Binary_Labels']

## Splits

In [15]:
X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, _, _ = train_test_split(X_train, X_train['Binary_Labels'], test_size=0.2, random_state=42)

In [16]:
# y_train = np.concatenate(X_train['Over_Labels'].values)
y_train = np.concatenate(X_train['Binary_Labels'].values)
y_test = np.concatenate(X_test['Binary_Labels'].values)
y_val = np.concatenate(X_val['Binary_Labels'].values)

ids_test = np.concatenate(X_test['IDS'].values)

# w_train = np.concatenate(X_train['Over_Weights'].values)
w_train = np.concatenate(X_train['Weights'].values)
w_test = np.concatenate(X_test['Weights'].values)
w_val = np.concatenate(X_val['Weights'].values)

# X_train = np.concatenate(X_train['Oversamples'].values)
X_train = np.concatenate(X_train['Sub_Divisions'].values)
X_test = np.concatenate(X_test['Sub_Divisions'].values)
X_val = np.concatenate(X_val['Sub_Divisions'].values)

## Show class imbalance

In [17]:
Counter(y_train)

Counter({1.0: 566, 0.0: 5654})

## Calculate Initial Bias

In [18]:
def calculate_bias(y):
    count = np.bincount(y.astype('int64'))
    neg, pos = count[0], count[1]
    total = neg + pos
    return np.log([pos/neg])

## Load model

In [19]:
def build_model(initial_bias, rand):
    initializer = tf.keras.initializers.HeNormal()
    bias_initializer = tf.keras.initializers.Constant(initial_bias)
    initializer = tf.keras.initializers.HeNormal()
    neurons = [[32, 64, 256], [64, 128, 256], [32, 64, 512], [8, 16, 128], [64, 128, 256],]
    first_neuron, second_neuron, third_neuron = neurons[rand]
    print(first_neuron, second_neuron, third_neuron)
    return Sequential([
        Input(shape=(N_MELS, 4, 33)),
        Conv2D(first_neuron, 8, activation='relu', kernel_initializer=initializer, padding="same"),
        MaxPooling2D(pool_size=(5, 2)),
        BatchNormalization(),
        Dropout(0.5),
        Conv2D(second_neuron, 6, activation='relu', padding="same", kernel_initializer=initializer),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),
        Dropout(0.5),
        Flatten(),
        Dense(third_neuron, activation='sigmoid'),
        Dropout(0.5),
        Dense(1, activation='sigmoid', bias_initializer=bias_initializer)
    ])

## DIY Bagging

In [20]:
def get_bootstrap_sample(X_train, y_train, w_train):
    n = X_train.shape[0]
    samples = np.random.choice(n, size=n)
    return X_train[samples, :, :, :], y_train[samples], w_train[samples]

## Train model

In [21]:
M = 5
models = [None] * M
for m in range(M):
    X, y, w = get_bootstrap_sample(X_train, y_train, w_train)
    bias = calculate_bias(y)
    r = np.random.randint(5)
    models[m] = build_model(bias, r)
    sgd = tf.keras.optimizers.SGD(lr=0.05, decay=1e-4, momentum=0.9, nesterov=True)
    models[m].compile(loss='binary_crossentropy', optimizer=sgd, metrics=get_metrics())
    print(f'Training model {m}')
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=20, mode='max', restore_best_weights=True)
    models[m].fit(
      X, 
      y, 
      batch_size=128,
      epochs=80, 
      shuffle=True,
      verbose=1,
      sample_weight=w,
      validation_data=(X_val, y_val),
      callbacks=[early_stopping]
    )
    print('')

32 64 256
Training model 0
Epoch 1/80
49/49 [==============================] - 11s 202ms/step - loss: 0.6646 - accuracy: 0.4239 - precision: 0.1330 - recall: 0.1361 - auc: 0.5635 - val_loss: 0.4195 - val_accuracy: 0.4450 - val_precision: 0.2258 - val_recall: 0.4118 - val_auc: 0.7269
Epoch 2/80
49/49 [==============================] - 9s 179ms/step - loss: 0.5250 - accuracy: 0.6371 - precision: 0.2954 - recall: 0.4062 - auc: 0.7837 - val_loss: 0.6495 - val_accuracy: 0.4337 - val_precision: 0.1877 - val_recall: 0.7868 - val_auc: 0.8172
Epoch 3/80
49/49 [==============================] - 9s 180ms/step - loss: 0.3731 - accuracy: 0.7384 - precision: 0.4124 - recall: 0.6159 - auc: 0.8866 - val_loss: 0.3810 - val_accuracy: 0.6664 - val_precision: 0.3116 - val_recall: 0.6691 - val_auc: 0.8636
Epoch 4/80
49/49 [==============================] - 9s 182ms/step - loss: 0.3285 - accuracy: 0.7847 - precision: 0.4712 - recall: 0.7036 - auc: 0.9179 - val_loss: 0.2988 - val_accuracy: 0.7858 - val_preci

## Calculate predictions

In [22]:
votes = np.zeros((M, X_test.shape[0]))
for m in range(M):
    m_pred = models[m].predict(X_test, batch_size=1)
    votes[m] = m_pred.squeeze()

In [23]:
preds = votes.sum(axis=0)/5

In [24]:
for m in range(M):
    models[m].evaluate(X_test, y_test)

69/69 [==============================] - 1s 12ms/step - loss: 0.3390 - accuracy: 0.8441 - precision: 0.3850 - recall: 0.4915 - auc: 0.8076


In [25]:
preds_2 = models[4].predict(X_test, batch_size=1)

## Print classification report

In [26]:
y_pred = [1 if (p > 0.5) else 0 for p in preds]
y_pred = np.asarray(y_pred)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95      2023
         1.0       0.47      0.45      0.46       177

    accuracy                           0.91      2200
   macro avg       0.71      0.70      0.71      2200
weighted avg       0.91      0.91      0.91      2200



## Calculate F1-Score

In [27]:
f_score, precision, recall = evaluate(harmonix_beats, preds_2, ids_test, True)

print("F-score: ", f_score)
print("Precision: ", precision)
print("Recall: ", recall)

F-score:  0.6016844746407315
Precision:  0.5984541847041847
Recall:  0.6291569833675096


## Save Model

## Save Results

In [28]:
# np.savez(os.path.join(ROOT, 'data', '06_Results', 'harmonix_voting.npz'), preds=preds, ids=ids_test)